# Practical session: first steps with PyCuda


In this assignment, you will learn about CUDA through the PyCuda language. Pycuda is a Python interface to the CUDA language, and more specifically a wrapper around C functions.

Online documentation is available at this address: : https://documen.tician.de/pycuda/

Before you start, you need to enable GPU support in Collab. To do this go to :
 - Edit->Notebook Settings

and activate the GPU (Hardware acceleration).

The following cell allows you to install PyCuda in your Collab work environment

In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=ad71ea2aa6f50d9d683b7e2b87bc8eca9e87f9047948eed756c71cdecd659aa7
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [ ]:
pip install scikit-cuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 2.1 MB/s eta 0:00:00


Here is a simple example (a kernel that multiplies term by term the elements of a table), from the PyCuda documentation. Before starting, test its execution and check the output of the program.

In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy

from pycuda.compiler import SourceModule
mod = SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = numpy.random.randn(400).astype(numpy.float32)
b = numpy.random.randn(400).astype(numpy.float32)

dest = numpy.zeros_like(a)
multiply_them(
        drv.Out(dest), drv.In(a), drv.In(b),
        block=(400,1,1), grid=(1,1))

print(dest-a*b)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

Try to design the same code with `GPUarray`


In [ ]:
from pycuda.gpuarray import GPUArray
import numpy as np
import time


mod = SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = np.random.randn(400).astype(np.float32)
b = np.random.randn(400).astype(np.float32)

dest_gpu = GPUArray(a.shape, np.float32)
a_gpu = GPUArray(a.shape, np.float32)
b_gpu = GPUArray(b.shape, np.float32)

dest_gpu.set(a * b)
a_gpu.set(a)
b_gpu.set(b)

multiply_them(dest_gpu, a_gpu, b_gpu, block=(400, 1, 1), grid=(1, 1))

dest_gpu_result = np.empty_like(a)
dest_gpu.get(dest_gpu_result)

print(dest_gpu_result - a * b)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.


# First part
In the first part of this session, we will see how to define a very simple kernel, allowing to invert the contents of a table.



In [ ]:
N = 100000
my_tab = numpy.arange(N)
# print(my_tab)
# my_tab[-1]

First code your kernel function


Then test it:

In [ ]:
N = 100000
my_tab = numpy.arange(N, dtype=numpy.int32)

mod = SourceModule("""
__global__ void invert_tab(int *my_tab, int N) {
    int idx = threadIdx.x + blockDim.x * blockIdx.x;

    if (idx < N) {
        my_tab[idx] = N - 1 - my_tab[idx];
    }
}
""")

invert_tab = mod.get_function("invert_tab")

# GPU memory
my_tab_gpu = drv.mem_alloc(my_tab.nbytes)

# Transfer my_tab to GPU
drv.memcpy_htod(my_tab_gpu, my_tab)

# Configure kernel
block_size = 256
grid_size = (N + block_size - 1) // block_size
invert_tab(my_tab_gpu, numpy.int32(N), block=(block_size, 1, 1), grid=(grid_size, 1))

# Copy the result back to CPU
drv.memcpy_dtoh(my_tab, my_tab_gpu)

# Print two arrays
print("Original Array:")
print(numpy.arange(N))
print("\nInverted Array:")
print(my_tab)

# Test
expected_result = numpy.arange(N, dtype=numpy.int32)[::-1]
assert (my_tab == expected_result).all(), "Test failed!"

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


Original Array:
[    0     1     2 ... 99997 99998 99999]

Inverted Array:
[99999 99998 99997 ...     2     1     0]


# Second part
On the same basis, now write a kernel that takes a matrix of data of size NxN, and calculates a vector of size N which is the sum of each line of the matrix.

You will take a grid of size (4,4,1)

Test.

In [ ]:
N = 4
matrix = numpy.random.randn(N, N).astype(numpy.float32)

mod = SourceModule("""
__global__ void sum_rows(float *matrix, float *sum_vector, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (row < N) {
        float sum = 0.0f;
        for (int i = 0; i < N; ++i) {
            sum += matrix[row * N + i];
        }
        sum_vector[row] = sum;
    }
}
""")

sum_rows = mod.get_function("sum_rows")

# GPU memory
matrix_gpu = drv.mem_alloc(matrix.nbytes)
sum_vector_gpu = drv.mem_alloc(N * numpy.float32().itemsize)

# Transfer data to GPU
drv.memcpy_htod(matrix_gpu, matrix)

# Configure kernel
block_size = (1, 4, 1)
grid_size = (1, (N + block_size[1] - 1) // block_size[1], 1)
sum_rows(matrix_gpu, sum_vector_gpu, numpy.int32(N), block=block_size, grid=grid_size)

# Copy the result back to CPU
sum_vector = numpy.empty_like(matrix[:, 0], dtype=numpy.float32)
drv.memcpy_dtoh(sum_vector, sum_vector_gpu)

# Test
expected_sum_vector = numpy.sum(matrix, axis=1)
assert (sum_vector == expected_sum_vector).all(), f"Assertion failed: {sum_vector} vs {expected_sum_vector}"

# Third part
Try to implement a kernel that computes the product of two square matrices, as seen in the course. Compare its execution time with the same  product written in numpy. For what size of the matrix do you get acceleration ?


In [ ]:
def matrix_multiply_cpu(a, b):
    return numpy.dot(a, b)

N = 1000

# Generate random matrices
matrix_a = numpy.random.randn(N, N).astype(numpy.float32)
matrix_b = numpy.random.randn(N, N).astype(numpy.float32)

# PyCUDA kernel for matrix multiplication
mod = SourceModule("""
__global__ void matrix_multiply(float *a, float *b, float *c, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < N; ++k) {
            sum += a[row * N + k] * b[k * N + col];
        }
        c[row * N + col] = sum;
    }
}
""")

matrix_multiply_gpu = mod.get_function("matrix_multiply")

# Allocate GPU memory
matrix_a_gpu = drv.mem_alloc(matrix_a.nbytes)
matrix_b_gpu = drv.mem_alloc(matrix_b.nbytes)
result_gpu = drv.mem_alloc(matrix_a.nbytes)

# Transfer data to GPU
drv.memcpy_htod(matrix_a_gpu, matrix_a)
drv.memcpy_htod(matrix_b_gpu, matrix_b)

# Configure kernel
block_size = (16, 16, 1)
grid_size = ((N + block_size[0] - 1) // block_size[0], (N + block_size[1] - 1) // block_size[1], 1)

start_time = time.time()
matrix_multiply_gpu(matrix_a_gpu, matrix_b_gpu, result_gpu, numpy.int32(N), block=block_size, grid=grid_size)
drv.Context.synchronize()
gpu_time = time.time() - start_time

# Copy the result back to CPU
result_gpu_matrix = numpy.empty_like(matrix_a)
drv.memcpy_dtoh(result_gpu_matrix, result_gpu)

# Perform CPU matrix multiplication
start_time = time.time()
result_cpu_matrix = matrix_multiply_cpu(matrix_a, matrix_b)
cpu_time = time.time() - start_time

print("GPU Result:")
print(result_gpu_matrix)
print("\nCPU Result:")
print(result_cpu_matrix)


epsilon = 1e-3  # Tolerance level
assert numpy.allclose(result_gpu_matrix, result_cpu_matrix, rtol=epsilon, atol=epsilon), "Results do not match!"



print(f"Matrix size: {N}x{N}")
print(f"GPU Time: {gpu_time:.5f} seconds")
print(f"CPU Time: {cpu_time:.5f} seconds")
print(f"Speedup: {cpu_time / gpu_time:.2f}x")


GPU Result:
[[ 11.871438    8.589899   27.330284  ...  -9.542024   12.873382
   12.727498 ]
 [ 21.745056   14.524875  -10.145751  ...  -2.4736772 -55.797756
   25.716566 ]
 [-29.202597   -5.1413164 -10.481479  ... -27.710413   16.680696
   22.223158 ]
 ...
 [-24.316147   58.34827   -16.907825  ... -40.165874   -1.0559348
   -2.329887 ]
 [ 13.075979  -55.709743   18.962326  ... -21.105322   18.362955
  -15.534467 ]
 [  7.102917   13.199413  -47.24878   ... -55.80871    -4.2112136
   20.100468 ]]

CPU Result:
[[ 11.871475    8.589907   27.330263  ...  -9.5420265  12.873375
   12.727499 ]
 [ 21.745062   14.524878  -10.145751  ...  -2.4736814 -55.797737
   25.716564 ]
 [-29.202595   -5.1413136 -10.481472  ... -27.710423   16.680693
   22.223145 ]
 ...
 [-24.316147   58.348267  -16.907822  ... -40.1659     -1.0559444
   -2.3298893]
 [ 13.07598   -55.709747   18.96231   ... -21.105326   18.362963
  -15.534463 ]
 [  7.1029134  13.199425  -47.248802  ... -55.80867    -4.211193
   20.100458 ]]
